# Testing for E1 - COGNITION 7Subjects
fistly define the UUID of the experiment to use

In [44]:
experimentId="ee715b34-ba07-49f0-9102-9652bb173c6c"

-------------------------------------------------------

In [45]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import sklearn.metrics as skm

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"\\COGNITION"
dreamspath=datapath+"\\DREAMS"
masspath=datapath+"\\MASS"


## load cognition

In [46]:
samplerate=200  #Should rethink this

In [47]:
annotations, signalsMetadata = loadCOGNITIONSpindles(cognipath)

In [48]:
annotations.head()

,subjectId,channel,startTime,stopTime,duration,phase,labelerId,type,samplerate,startInd,stopInd
0,0001,C3,1938.132297,1939.202326,1.070029,2,0001,spindle,200,387626,387840
1,0001,C3,2047.217898,2048.112834,0.894936,2,0001,spindle,200,409444,409623
2,0001,C3,2170.856032,2171.634237,0.778205,3,0001,spindle,200,434171,434327
3,0001,C3,2219.766538,2220.894943,1.128405,3,0001,spindle,200,443953,444179
4,0001,C3,2263.482492,2264.357974,0.875482,3,0001,spindle,200,452696,452872


In [49]:
signalsMetadata.head()

,filename,channel,subjectId,duration,samplerate,isOriginalSamplerate,database
0,COG001_canal_C3.txt,C3-M2,0001,39600,200,False,COGNITION
1,COG002_canal_C3.txt,C3-M2,0002,39600,200,False,COGNITION
2,COG003_canal_C3.txt,C3-M2,0003,39600,200,False,COGNITION
3,COG004_canal_C3.txt,C3-M2,0004,39600,200,False,COGNITION
4,COG005_canal_C3.txt,C3-M2,0005,39600,200,False,COGNITION


## Load experiment results

In [50]:
experimentModels, featureSelection = loadExperiment(experimentId,datapath)

In [51]:
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0004, 0005, 0006, 0007]","[0003, 0008]",0002,a4659423-83e4-4da3-91b5-177fa8bc17fe,0.008075
1,0001,E1,[0001],"[0002, 0005, 0006, 0008]","[0007, 0004]",0003,ad3be387-5be4-4e8e-b57d-072003b617d3,0.005000
2,0001,E1,[0001],"[0002, 0003, 0005, 0007]","[0006, 0008]",0004,8bcbbc8b-b83e-4b97-8f7c-0dd0760680e7,0.012035
3,0001,E1,[0001],"[0002, 0003, 0007, 0008]","[0006, 0004]",0005,f88ed731-de3a-4c9a-90d5-04d1fcd93d45,0.010340
4,0001,E1,[0001],"[0002, 0003, 0007, 0008]","[0004, 0005]",0006,16a691da-d735-4ad2-953c-db2678980107,0.010340
5,0001,E1,[0001],"[0002, 0004, 0005, 0008]","[0003, 0006]",0007,d91f41ff-0308-46df-9bcd-67d46f377777,0.006673
6,0001,E1,[0001],"[0002, 0005, 0006, 0007]","[0003, 0004]",0008,a1eacac2-9e31-437a-9350-110d2ef2f1a5,0.006822


In [52]:
#we show the difference in class inbalance for the annotation criteria considered
experimentModels[['criteriumName','spindleTimeRate']].groupby('criteriumName').describe()

spindleTimeRate                                                 \
                        count      mean       std    min       25%       50%   
criteriumName                                                                  
E1                        7.0  0.008469  0.002511  0.005  0.006748  0.008075   

                                  
                   75%       max  
criteriumName                     
E1             0.01034  0.012035

In [53]:
featureSelection

,window,characteristic,bandName,Times_selected
0,2,sigmaIndex,broadband,1000.0
1,2,hjortActivity,sigma,1000.0
2,1.5,sigmaIndex,broadband,1000.0
3,1,hjortActivity,sigma,1000.0
4,1,sigmaIndex,broadband,1000.0
5,2,hjortActivity,beta1,1000.0
6,2,hjortActivity,beta2,1000.0
7,1.5,hjortActivity,sigma,999.0
8,2,hjortMobility,beta1,994.0
9,2,hjortActivity,delta2,986.0


## Hyperparameter definition
this should come from a previous evaluation notebook

In [54]:
hyperClose=0.1
hyperDuration=0.3
hyperThres=0.3
hyperDepth=10

## Testing with E1 criterium
we test the optimal points for the prediction threshold and number of boost iterations in the different validation groups

In [55]:
experimentModels=experimentModels[experimentModels.criteriumName=='E1'].reset_index(drop=True)
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0004, 0005, 0006, 0007]","[0003, 0008]",0002,a4659423-83e4-4da3-91b5-177fa8bc17fe,0.008075
1,0001,E1,[0001],"[0002, 0005, 0006, 0008]","[0007, 0004]",0003,ad3be387-5be4-4e8e-b57d-072003b617d3,0.005000
2,0001,E1,[0001],"[0002, 0003, 0005, 0007]","[0006, 0008]",0004,8bcbbc8b-b83e-4b97-8f7c-0dd0760680e7,0.012035
3,0001,E1,[0001],"[0002, 0003, 0007, 0008]","[0006, 0004]",0005,f88ed731-de3a-4c9a-90d5-04d1fcd93d45,0.010340
4,0001,E1,[0001],"[0002, 0003, 0007, 0008]","[0004, 0005]",0006,16a691da-d735-4ad2-953c-db2678980107,0.010340
5,0001,E1,[0001],"[0002, 0004, 0005, 0008]","[0003, 0006]",0007,d91f41ff-0308-46df-9bcd-67d46f377777,0.006673
6,0001,E1,[0001],"[0002, 0005, 0006, 0007]","[0003, 0004]",0008,a1eacac2-9e31-437a-9350-110d2ef2f1a5,0.006822


In [56]:
rawF1s=[]
rawPrecisions=[]
rawRecalls=[]

f1s=[]
precisions=[]
recalls=[]

eventF1s=[]
eventPrecisions=[]
eventRecalls=[]

checks=[]

for ind,row in experimentModels.iterrows():
    print('*************************')
    print(str(ind+1)+' of '+str(len(experimentModels)) )
    #load model
    model=loadBooster(row.modelId,experimentId,datapath)

    testSubjectId=row.test
    #Define annotations criterium
    usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Load features and labels
    testFeatures=loadFeatureMatrix([testSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
    testLabels=loadLabelsVector([testSubjectId],usedAnnotations,signalsMetadata,samplerate)

    #Predict
    testDMatrix=xgb.DMatrix(data=testFeatures)
    probabilities=model.predict(testDMatrix,iteration_range=(0,hyperDepth))
    rawLabels=probabilities>=hyperThres
    #Raw Metrics
    rawTp=np.sum(rawLabels*testLabels)
    rawFp=np.sum(rawLabels*(1-testLabels))
    rawTn=np.sum((1-rawLabels)*(1-testLabels))
    rawFn=np.sum((1-rawLabels)*testLabels)
    #Raw appends
    rawF1s.append(2*rawTp/(2*rawTp+rawFp+rawFn))
    rawPrecisions.append(rawTp/(rawTp+rawFp) )
    rawRecalls.append(rawTp/(rawTp+rawFn))
    #Process
    processedLabels=labelingProcess(rawLabels,hyperClose,hyperDuration,samplerate)
    #Processed metrics
    tp=np.sum(processedLabels*testLabels)
    fp=np.sum(processedLabels*(1-testLabels))
    tn=np.sum((1-processedLabels)*(1-testLabels))
    fn=np.sum((1-processedLabels)*testLabels)
    #Processed appends
    f1s.append(2*tp/(2*tp+fp+fn))
    precisions.append(tp/(tp+fp))
    recalls.append(tp/(tp+fn))

    #By-event metrics
    processedAnnotations=labelVectorToAnnotations(processedLabels,samplerate)
    processedAnnotations=processedAnnotations[processedAnnotations.duration<5]
    gtAnnotations=labelVectorToAnnotations(testLabels,samplerate)   #<- or just filter the annotations
    f,r,p=annotationPairToMetrics(gtAnnotations,processedAnnotations, thresIoU=0.3)
    
    #calculate metrics
    eventF1s.append(f)
    eventPrecisions.append(p)
    eventRecalls.append(r)

#include metrics in the dataframe
experimentModels['rawF1']=rawF1s
experimentModels['rawPrecision']=rawPrecisions
experimentModels['rawRecall']=rawRecalls

experimentModels['f1']=f1s
experimentModels['precision']=precisions
experimentModels['recall']=recalls

experimentModels['eventF1']=eventF1s
experimentModels['eventPrecision']=eventPrecisions
experimentModels['eventRecall']=eventRecalls


*************************
1 of 7
*************************
2 of 7
*************************
3 of 7
*************************
4 of 7
*************************
5 of 7
*************************
6 of 7
*************************
7 of 7


In [57]:
dumpPickle('experimentModelsTest_E1_temp.pkl',experimentModels)

In [58]:
experimentModels.columns

Index(['criteriumId', 'criteriumName', 'labelerIdList', 'train', 'val', 'test',
       'modelId', 'spindleTimeRate', 'rawF1', 'rawPrecision', 'rawRecall',
       'f1', 'precision', 'recall', 'eventF1', 'eventPrecision',
       'eventRecall'],
      dtype='object')

In [59]:
fig=px.scatter(experimentModels,x='f1',y='eventF1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['f1'], y=experimentModels['f1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [60]:
fig=px.scatter(experimentModels,x='rawF1',y='f1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [61]:
fig=px.scatter(experimentModels,x='rawF1',y='eventF1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [62]:
fig=px.scatter(experimentModels,x='eventF1',y='eventPrecision',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [63]:
fig=px.scatter(experimentModels,x='eventF1',y='eventRecall',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [64]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test').describe(percentiles=[0.5])

eventF1                                             eventPrecision  \
       count      mean std       min       50%       max          count   
test                                                                      
0002     1.0  0.637076 NaN  0.637076  0.637076  0.637076            1.0   
0003     1.0  0.848108 NaN  0.848108  0.848108  0.848108            1.0   
0004     1.0  0.778589 NaN  0.778589  0.778589  0.778589            1.0   
0005     1.0  0.652505 NaN  0.652505  0.652505  0.652505            1.0   
0006     1.0  0.684803 NaN  0.684803  0.684803  0.684803            1.0   
0007     1.0  0.655754 NaN  0.655754  0.655754  0.655754            1.0   
0008     1.0  0.454545 NaN  0.454545  0.454545  0.454545            1.0   

                                                 eventRecall                \
          mean std       min       50%       max       count      mean std   
test                                                                         
0002  0.853147 NaN  0.853147  0.853147  0.853147         1.0  0.508333 NaN   
0003  0.784216 NaN  0.784216  0.784216  0.784216         1.0  0.923439 NaN   
0004  0.794045 NaN  0.794045  0.794045  0.794045         1.0  0.763723 NaN   
0005  0.498339 NaN  0.498339  0.498339  0.498339         1.0  0.946203 NaN   
0006  0.596091 NaN  0.596091  0.596091  0.596091         1.0  0.805310 NaN   
0007  0.520440 NaN  0.520440  0.520440  0.520440         1.0  0.887097 NaN   
0008  0.789474 NaN  0.789474  0.789474  0.789474         1.0  0.319149 NaN   

                                    
           min       50%       max  
test                                
0002  0.508333  0.508333  0.508333  
0003  0.923439  0.923439  0.923439  
0004  0.763723  0.763723  0.763723  
0005  0.946203  0.946203  0.946203  
0006  0.805310  0.805310  0.805310  
0007  0.887097  0.887097  0.887097  
0008  0.319149  0.319149  0.319149

by-event

In [65]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean()

,test,eventF1,eventPrecision,eventRecall
0,0002,0.637076,0.853147,0.508333
1,0003,0.848108,0.784216,0.923439
2,0004,0.778589,0.794045,0.763723
3,0005,0.652505,0.498339,0.946203
4,0006,0.684803,0.596091,0.805310
5,0007,0.655754,0.520440,0.887097
6,0008,0.454545,0.789474,0.319149


In [66]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              2.857571e+23
eventF1           6.730544e-01
eventPrecision    6.908216e-01
eventRecall       7.361791e-01
dtype: float64

In [67]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().std()

C:\Users\GIB\AppData\Local\Temp\ipykernel_13108\2599554826.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.123613
eventPrecision    0.147486
eventRecall       0.235697
dtype: float64

by-sample

In [68]:
experimentModels[['test','f1','precision','recall']].groupby('test',as_index=False).mean().mean()

test         2.857571e+23
f1           6.238558e-01
precision    6.778400e-01
recall       6.587392e-01
dtype: float64

In [69]:
experimentModels[['test','f1','precision','recall']].groupby('test',as_index=False).mean().std()

C:\Users\GIB\AppData\Local\Temp\ipykernel_13108\319227537.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



f1           0.111684
precision    0.159263
recall       0.222091
dtype: float64

In [70]:
auxPrecision=pd.DataFrame({
    'metric':'event precision',
    'value':experimentModels.eventPrecision,
    'event F1':experimentModels.eventF1
})

auxRecall=pd.DataFrame({
    'metric':'event recall',
    'value':experimentModels.eventRecall,
    'event F1':experimentModels.eventF1
})
visualTradeoff=pd.concat((auxPrecision,auxRecall))

In [71]:
fig=px.scatter(visualTradeoff,x='event F1',y='value',color='metric', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

### Restricted to N2

In [72]:
rawF1s=[]
rawPrecisions=[]
rawRecalls=[]

f1s=[]
precisions=[]
recalls=[]

eventF1s=[]
eventPrecisions=[]
eventRecalls=[]

checks=[]

for ind,row in experimentModels.iterrows():
    print('*************************')
    print(str(ind+1)+' of '+str(len(experimentModels)) )
    #load model
    model=loadBooster(row.modelId,experimentId,datapath)
    testSubjectId=row.test
   
    #Define annotations criterium
    usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Load features and labels
    testFeatures=loadFeatureMatrix([testSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
    testLabels=loadLabelsVector([testSubjectId],usedAnnotations,signalsMetadata,samplerate)

    ##################################################
    hypnogram=loadCOGNITIONHypnogram(testSubjectId,cognipath)
    testLabels=testLabels*(hypnogram==2)
    ##################################################

    #Predict
    testDMatrix=xgb.DMatrix(data=testFeatures)
    probabilities=model.predict(testDMatrix,iteration_range=(0,hyperDepth))
    rawLabels=probabilities>=hyperThres

    ##################################################
    rawLabels=rawLabels*(hypnogram==2)
    ##################################################

    #Raw Metrics
    rawTp=np.sum(rawLabels*testLabels)
    rawFp=np.sum(rawLabels*(1-testLabels))
    rawTn=np.sum((1-rawLabels)*(1-testLabels))
    rawFn=np.sum((1-rawLabels)*testLabels)
    #Raw appends
    rawF1s.append(2*rawTp/(2*rawTp+rawFp+rawFn))
    rawPrecisions.append(rawTp/(rawTp+rawFp) )
    rawRecalls.append(rawTp/(rawTp+rawFn))
    #Process
    processedLabels=labelingProcess(rawLabels,hyperClose,hyperDuration,samplerate)
    #Processed metrics
    tp=np.sum(processedLabels*testLabels)
    fp=np.sum(processedLabels*(1-testLabels))
    tn=np.sum((1-processedLabels)*(1-testLabels))
    fn=np.sum((1-processedLabels)*testLabels)
    #Processed appends
    f1s.append(2*tp/(2*tp+fp+fn))
    precisions.append(tp/(tp+fp))
    recalls.append(tp/(tp+fn))

    #By-event metrics
    processedAnnotations=labelVectorToAnnotations(processedLabels,samplerate)
    processedAnnotations=processedAnnotations[processedAnnotations.duration<5]
    gtAnnotations=labelVectorToAnnotations(testLabels,samplerate)   #<- or just filter the annotations
    f,r,p=annotationPairToMetrics(gtAnnotations,processedAnnotations, thresIoU=0.3)
    
    #calculate metrics
    eventF1s.append(f)
    eventPrecisions.append(p)
    eventRecalls.append(r)

#include metrics in the dataframe
experimentModels['rawF1']=rawF1s
experimentModels['rawPrecision']=rawPrecisions
experimentModels['rawRecall']=rawRecalls

experimentModels['f1']=f1s
experimentModels['precision']=precisions
experimentModels['recall']=recalls

experimentModels['eventF1']=eventF1s
experimentModels['eventPrecision']=eventPrecisions
experimentModels['eventRecall']=eventRecalls


*************************
1 of 7
*************************
2 of 7
*************************
3 of 7
*************************
4 of 7
*************************
5 of 7
*************************
6 of 7
*************************
7 of 7


In [73]:
dumpPickle('experimentModelsTestN2_E1_temp.pkl',experimentModels)

In [74]:
experimentModels.columns

Index(['criteriumId', 'criteriumName', 'labelerIdList', 'train', 'val', 'test',
       'modelId', 'spindleTimeRate', 'rawF1', 'rawPrecision', 'rawRecall',
       'f1', 'precision', 'recall', 'eventF1', 'eventPrecision',
       'eventRecall'],
      dtype='object')

In [ ]:
fig=px.scatter(experimentModels,x='f1',y='eventF1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['f1'], y=experimentModels['f1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [75]:
fig=px.scatter(experimentModels,x='rawF1',y='f1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [76]:
fig=px.scatter(experimentModels,x='rawF1',y='eventF1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [77]:
fig=px.scatter(experimentModels,x='eventF1',y='eventPrecision',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [78]:
fig=px.scatter(experimentModels,x='eventF1',y='eventRecall',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [79]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test').describe(percentiles=[0.5])

eventF1                                             eventPrecision  \
       count      mean std       min       50%       max          count   
test                                                                      
0002     1.0  0.666667 NaN  0.666667  0.666667  0.666667            1.0   
0003     1.0  0.859840 NaN  0.859840  0.859840  0.859840            1.0   
0004     1.0  0.789610 NaN  0.789610  0.789610  0.789610            1.0   
0005     1.0  0.652125 NaN  0.652125  0.652125  0.652125            1.0   
0006     1.0  0.696970 NaN  0.696970  0.696970  0.696970            1.0   
0007     1.0  0.670478 NaN  0.670478  0.670478  0.670478            1.0   
0008     1.0  0.476190 NaN  0.476190  0.476190  0.476190            1.0   

                                                 eventRecall                \
          mean std       min       50%       max       count      mean std   
test                                                                         
0002  0.860656 NaN  0.860656  0.860656  0.860656         1.0  0.544041 NaN   
0003  0.803419 NaN  0.803419  0.803419  0.803419         1.0  0.924877 NaN   
0004  0.797900 NaN  0.797900  0.797900  0.797900         1.0  0.781491 NaN   
0005  0.496599 NaN  0.496599  0.496599  0.496599         1.0  0.950980 NaN   
0006  0.611307 NaN  0.611307  0.611307  0.611307         1.0  0.811321 NaN   
0007  0.537438 NaN  0.537438  0.537438  0.537438         1.0  0.891967 NaN   
0008  0.789474 NaN  0.789474  0.789474  0.789474         1.0  0.340909 NaN   

                                    
           min       50%       max  
test                                
0002  0.544041  0.544041  0.544041  
0003  0.924877  0.924877  0.924877  
0004  0.781491  0.781491  0.781491  
0005  0.950980  0.950980  0.950980  
0006  0.811321  0.811321  0.811321  
0007  0.891967  0.891967  0.891967  
0008  0.340909  0.340909  0.340909

by-event

In [80]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean()

,test,eventF1,eventPrecision,eventRecall
0,0002,0.666667,0.860656,0.544041
1,0003,0.859840,0.803419,0.924877
2,0004,0.789610,0.797900,0.781491
3,0005,0.652125,0.496599,0.950980
4,0006,0.696970,0.611307,0.811321
5,0007,0.670478,0.537438,0.891967
6,0008,0.476190,0.789474,0.340909


In [81]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              2.857571e+23
eventF1           6.874115e-01
eventPrecision    6.995417e-01
eventRecall       7.493695e-01
dtype: float64

In [82]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().std()

C:\Users\GIB\AppData\Local\Temp\ipykernel_13108\2599554826.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.120308
eventPrecision    0.147060
eventRecall       0.225704
dtype: float64

by-sample

In [83]:
experimentModels[['test','f1','precision','recall']].groupby('test',as_index=False).mean().mean()

test         2.857571e+23
f1           6.367077e-01
precision    6.827891e-01
recall       6.713402e-01
dtype: float64

In [84]:
experimentModels[['test','f1','precision','recall']].groupby('test',as_index=False).mean().std()

C:\Users\GIB\AppData\Local\Temp\ipykernel_13108\319227537.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



f1           0.105761
precision    0.156370
recall       0.212506
dtype: float64

In [85]:
auxPrecision=pd.DataFrame({
    'metric':'event precision',
    'value':experimentModels.eventPrecision,
    'event F1':experimentModels.eventF1
})

auxRecall=pd.DataFrame({
    'metric':'event recall',
    'value':experimentModels.eventRecall,
    'event F1':experimentModels.eventF1
})
visualTradeoff=pd.concat((auxPrecision,auxRecall))

In [86]:
fig=px.scatter(visualTradeoff,x='event F1',y='value',color='metric', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()